# Item-to-Item Collaborative Filtering

## Idea
Let $u$ be the active user and $i$ the referenced item
1. If $u$ liked items similar to $i$, he will probably like item $i$.
2. If he hated or disliked items similar to $i$, he will also hate item $i$.

The idea is therefore to look at how an active user $u$ rated items similar to $i$ to know how he would have rated item $i$

## Advantages over user-based CF

1. <b> Stability </b> : Items ratings are more stable than users ratings. New ratings on items are unlikely to significantly change the similarity between two items, particularly when the items have many ratings <a href="https://dl.acm.org/doi/10.1561/1100000009">(Michael D. Ekstrand, <i>et al.</i> 2011)</a>. 
2. <b> Scalability </b> : with stable item's ratings, it is reasonable to pre-compute similarities between items in an item-item similarity matrix (similarity between items can be computed offline). This will reduce the scalability concern of the algorithm. <a href="https://dl.acm.org/doi/10.1145/371920.372071">(Sarwar <i>et al.</i> 2001)</a>, <a href="https://dl.acm.org/doi/10.1561/1100000009">(Michael D. Ekstrand, <i>et al.</i> 2011)</a>.

## Algorithm : item-to-item collaborative filtering

The algorithm that defines item-based CF is described as follow <a href="http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.449.1171&rep=rep1&type=pdf">(B. Sarwar et al. 2001)</a><a href="https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.554.1671&rep=rep1&type=pdf">(George Karypis 2001)</a> :

1. First identify the $k$ most similar items for each item in the catalogue and record the corresponding similarities. To compute similarity between two items we can user the <i>Adjusted Cosine Similarity</i> that has proven to be more efficient than the basic <i>Cosine similarity measure</i> used for user-based collaborative as described in <a href="http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.449.1171&rep=rep1&type=pdf">(B. Sarwar et al. 2001)</a>. The Adjusted Cosine distance between two items $i$ and $j$ is computed as follow

<center>
$
\large
 w_{i,j}= \frac{\sum_{u\in U}(r_{u,i}-\bar{r}_u)(r_{u,j}-\bar{r}_u)}{\sqrt{\sum_{u\in U} (r_{u,i}-\bar{r}_u)^2}\sqrt{\sum_{u\in U} (r_{u,j}-\bar{r}_u)^2}}
$
</center>

$w_{i,j}$ is the degree of similarity between items $i$ and $j$. This term is computed for all users $u\in U$, where $U$ is the set of users that rated both items $i$ and $j$. Let's denote by $S^{(i)}$ the set of the $k$ most similar items to item $i$.


2. To produce top-N recommendations for a given user $u$ that has already purchased a set $I_u$ of items, do the following :

    a. Find the set $C$ of candidate items by taking the union of all $S^{(i)}, \forall i\in I_u$ and removing each of the items in the set $I_u$.
<center>
$
 C = \bigcup_{i\in I_u}\{S^{(i)}\}\smallsetminus I_u
$
</center>    
    b. $\forall c\in C$, compute similarity between c and the set $I_u$ as follows:
<center>
$
 w_{c,I_u} = \sum_{i\in I_u} w_{c,i}, \forall c \in C
$
</center>
    c. Sort items in $C$ in decreasing order of $w_{c,I_u}, \forall c \in C$, and return the first $N$ items as the Top-N recommendation list.

Before returning the first $N$ items as top-N recommendation list, we can make predictions about what user $u$ would have given to each items in the top-N recommendation list, rearrange the list in descending order of predicted ratings and return the rearranged list as the final recommendation list. Rating prediction for item-based CF is given by the following formular <a href="http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.449.1171&rep=rep1&type=pdf">(B. Sarwar et al. 2001)</a>:

<center>
$
\large
 \hat{r}_{u,i}=\frac{\sum_{i\in S^{(i)}}r_{u,j}\cdot w_{i,j}}{\sum_{j\in S^{(i)}}|w_{i,j}|}
$
</center>

### Import useful requirements

In [65]:
from tools.utils import load_ratings, load_movies, download_data
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

import pandas as pd
import numpy as np
import os

Download movielen data if it doesn't exist

In [27]:
data = os.path.join('tools', 'ml-latest-small')

if os.path.exists(data):
    print('Data already exists ...')
    ratings_csv, movies_csv = os.path.join(data, 'ratings.csv'), os.path.join(data, 'movies.csv')
else:
    ratings_csv, movies_csv = download_data()

Data already exists ...


### Load ratings

In [28]:
ratings, movies = load_ratings(ratings_csv), load_movies(movies_csv)

In [135]:
itemids = sorted(ratings['itemid'].unique())

In [165]:
itemids_to_idx = {itemid:idx for (itemid,idx) in zip(itemids, range(0, len(itemids)))}

In [218]:
idx_to_itemids = {idx:itemid for (idx,itemid) in zip(range(0,len(itemids)),itemids)}

In [243]:
ratings

,userid,itemid,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0


Let's implements the item-based collaborative filtering algorithm described above

### Step 1. Find similarities for each of the items

To compute similarity between two items $i$ and $j$, we need to :

1. find all users who rated both of them,
2. Normalize their ratings on items $i$ and $j$
3. Apply the cosine metric to the normalized ratings to compute similarity between $i$ and $j$

Function ```normalize()``` process the rating dataframe to normalize ratings of all users

In [358]:
def normalize():
    """
    
    """
    # compute mean rating for each user
    mean = ratings.groupby(by='userid', as_index=False)['rating'].mean()
    norm_ratings = pd.merge(ratings, mean, suffixes=('','_mean'), on='userid')
    
    # normalize each rating by substracting the mean rating of the corresponding user
    norm_ratings['norm_rating'] = norm_ratings['rating'] - norm_ratings['rating_mean']
    
    return mean, norm_ratings

In [359]:
mean, norm_ratings = normalize()
norm_ratings.head()

,userid,itemid,rating,rating_mean,norm_rating
0,1,1,4.0,4.366379,-0.366379
1,1,3,4.0,4.366379,-0.366379
2,1,6,4.0,4.366379,-0.366379
3,1,47,5.0,4.366379,0.633621
4,1,50,5.0,4.366379,0.633621


now that each rating has been normalized, we can represent each item by a vector of its normalized ratings

In [56]:
def item_representation(norm_ratings):
    """
    
    """
    
    df = pd.crosstab(mean_ratings.itemid, mean_ratings.userid, mean_ratings.norm_rating, aggfunc=sum)
    df = df.fillna(0)

    R = csr_matrix(df.values)
    
    return R, df

In [129]:
R, df = item_representation(norm_ratings)

In [58]:
df

userid,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,...,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610
itemid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,-0.366379,0.0,0.0,0.0,0.363636,0.000000,1.269737,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,-0.948148,0.0,0.290476,-0.232072,1.392603,0.000000,0.239278,0.0,0.0,0.0,0.0,0.0,-0.548148,0.0,0.0,0.0,1.08,-0.754902,-0.788462,0.0,0.0,0.0,0.0,0.0,0.0,1.23301,0.0,0.000000,0.447368,-0.354167,0.12406,1.0,0.0,0.0,0.0,0.219355,0.000000,0.0,0.0,-0.030303,0.0,0.0,1.607143,0.000000,0.0,0.0,0.0,0.000000,1.369004,0.231141,0.0,-0.02029,0.0,-0.73373,0.0,0.0,1.4,0.0,0.788095,0.0,0.0,-2.584034,0.0,0.836066,0.0,0.0,0.0,-0.881057,0.0,0.00000,0.0,0.071429,0.0,0.0,-0.472973,-1.074074,0.604348,0.0,-1.28866,0.000000,0.0,1.115385,0.0,0.586957,0.0,0.000000,0.0,0.000000,0.092838,0.000000,0.0,0.0,0.088235,0.0,0.0,0.0,0.000000,-0.592308,0.0,0.0,0.0,0.000000,0.000000,0.0,-0.676744,0.000000,0.706897,0.000000,0.0,0.01,0.000000,0.0,0.0,0.0,0.0,-0.535714,0.0,-1.038905,0.0,-0.571429,0.369176,0.0,0.021277,0.0,0.0,-0.499178,0.46131,0.0,0.0,-0.128906,1.652174,0.0,0.000,0.0,0.000000,0.000000,1.457627,0.0,-0.217877,0.0,-0.608696,0.307789,0.0,0.0,1.226804,1.291762,0.166667,0.000000,0.0,0.0,0.0,0.926316,0.060694,0.0,0.250929,0.00,0.365854,0.0,0.0,0.0,0.0,0.0,1.624447,-0.090909,0.217391,0.0,0.000000,0.488741,0.0,0.000000,0.468085,-0.079646,0.0,0.0,0.0,0.0,0.258824,0.0,-1.8,0.0,0.0,0.0,0.0,0.0,0.0,-0.30988,0.9,0.181141,0.0,0.0,0.0,0.96,0.0,0.0,0.0,0.0,0.0,0.0,-0.327381,0.136364,0.0,-0.664634,1.238173,0.0,0.33428,1.036232,0.000000,0.000,0.226667,0.0,0.0,0.023669,0.0,0.0,1.261538,0.000000,0.0,0.24942,-0.3,1.49505,0.0,0.0,0.0,0.0,-0.026882,1.085938,0.0,0.0,0.0,0.0,0.0,0.0,1.253333,0.0,0.303537,0.0,...,0.0,0.0,0.0,0.

```df``` is a cross table dataframe where rows are items and columns are users. Each row represents the vector of normalized ratings attributed to the corresponding item. Missing values are filled with zeros (0). ```R``` is the numpy array corresponding to ```df``` that will be fitted to a $k$-NN model.

Let's build and fit our $k$-NN model using sklearn

In [60]:
def create_model(R, k):
    """
    :param
        R : numpy array of item representations
        k : number of nearest neighbors to return
    
    :return
        model : our knn model
    """
    
    # create the nearest neighbors model
    model = NearestNeighbors(metric='cosine', n_neighbors=k+1, algorithm='brute')
    
    # fit the model with ratings
    model.fit(R)
    
    return model

Function ```nearest_neighbors()``` compute nearest neighbors

In [314]:
def nearest_neighbors(R, model):
    """
    compute the top 30 similar items for each item.
    
    :param
        - R : items representations
        - model : nearest neighbors model
    
    :return
        - similarities
        - neighbors
    """
    
    similarities, neighbors = model.kneighbors(R)
    
    similarities = 1 - np.squeeze(similarities)
    neighbors = np.squeeze(neighbors)
    
    for idx in range(similarities.shape[0]):
        similarities[idx][0] = 0
        
    return similarities, neighbors

Let's call functions ```create_model()``` and ```nearest_neighbors()``` to respectively create the $k$-NN model (with $k=30$) and compute the nearest neighbors for all items in our database

In [315]:
model = create_model(R, k=30)

In [316]:
similarities, neighbors = nearest_neighbors(R, model)

In [317]:
print('neighbors shape : ', neighbors.shape)
print('similarities shape : ', similarities.shape)

neighbors shape :  (9724, 31)
similarities shape :  (9724, 31)


```neighbors``` and ```similarities``` are numpy array, were each entries are list of $30$ neighbors with their corresponding similarities

### Step 2. Top N recommendation for a given user

Top-N recommendations are made for example for a user $u$ who has already rated a set of items $I_u$

#### 2.a- Finding candidate items

To find candidate items for user $u$, we need to :

1. Find the set $I_u$ of items already rated by user $u$,
2. Take the union of similar items as $C$ for all items in $I_u$
3. exclude from the set $C$ all items in $I_u$, to avoid recommend to a user items he has already purchased.

These are done in function ```candidate_items()```

In [219]:
def candidate_items(userid):
    """
    :param
        - userid : user id for which we wish to find candidate items
    
    :return
        - I_u : list of items already purchased by userid
        - candidates : list of candidate items
    """
    
    # 1. Finding the set I_u of items already rated by user userid
    I_u = ratings.loc[ratings.userid == userid].itemid.to_list()
    
    # 2. Taking the union of similar items for all items in I_u to form the set of candidate items
    C = set()
    
    for iid in I_u:        
        # get the index of item iid in the nearest neighbors set
        idx = itemids_to_idx[iid]
        
        # add the neighbors of item iid in the set of candidate items
        
        C.update([ idx_to_itemids[ix] for ix in neighbors[idx]])
        
    C = list(C)
    
    # 3. exclude from the set C all items in I_u.
    candidates = np.setdiff1d(C, I_u, assume_unique=True)
    
    return I_u, candidates

In [220]:
i_u, u_candidates = candidate_items(514)

In [221]:
print('number of items purchased by user 514 : ', len(i_u))
print('number of candidate items for user 514 : ', len(u_candidates))

number of items purchased by user 514 :  397
number of candidate items for user 514 :  3818


In [222]:
u_candidates

array([122882,      6,   8199, ..., 106489,   8189,  40959])

#### 2.b- Find similarity between each candidate item and the set $I_u$

In [178]:
def similarity_with_Iu(c, I_u):
    """
    compute similarity between an item c and a set of items I_u. For each item i in I_u, get similarity between 
    i and c, if c exists in the set of items similar to itemid
    
    :param
        - c : itemid of a candidate item
        - I_u : set of items already purchased by a given user
    
    :return
        - w : similarity between c and I_u
    """
    w = 0    
    for i in I_u :        
        # idx of item i in nearest neighbors set
        idx = itemids_to_idx[i]
        
        # get similarity between itemid and c, if c is one of the k nearest neighbors of itemid
        if c in neighbors[idx] :
            i_similarities = similarities[idx]
            w = w + i_similarities[list(neighbors[idx]).index(c)]
    
    return w

#### 2.c- Rank candidate items according to their similarities to $I_u$

In [188]:
def rank_candidates(candidates, I_u):
    """
    rank candidate items according to their similarities with I_u
    
    :param
        - candidates : list of candidate items
        - I_u : list of items purchased by the user
    
    :return:
        - ranked_candidates : dataframe of candidate items, ranked in descending order of similarities with I_u
    """
    
    # list of candidate items mapped to their corresponding similarities to I_u 
    mapping = []
    for c in candidates:
        # compute similarity between c and I_u
        w = similarity_with_Iu(c, I_u)
        mapping.append((c,w))

    ranked_candidates = pd.DataFrame(mapping, columns=['itemid','similarity_with_Iu'])
    
    # rank candidate items according to their similarities
    ranked_candidates = ranked_candidates.sort_values(by=['similarity_with_Iu'], ascending=False)
    
    return ranked_candidates
        

## Putting all together

Now that we defined all functions necessary to build our item to item top-N recommendation, let's define function ```item2item_topN()``` that makes top-$N$ recommendations for a given user 

In [203]:
def item2item_topN(userid, N=30):
    """
    Produce top-N recommendation for a given user
    
    :param
        - userid : user for which we produce top-N recommendation
        - N : length of the top-N recommendation list
    
    :return
        - topN_list
    """
    # find candidate items
    I_u, candidates = candidate_items(userid)
    
    # rank candidate items according to their similarities with I_u
    ranked_candidates = rank_candidates(candidates, I_u)
    
    # get the first N row of ranked_candidates to build the top N recommendation list
    topN_list = ranked_candidates.iloc[:N]
    
    topN_list = pd.merge(topN_list, movies, on='itemid', how='inner')
    
    return topN_list

In [226]:
topN = item2item_topN(514)

In [227]:
topN

,itemid,similarity_with_Iu,title
0,2470,50.154702,Crocodile Dundee (1986)
1,5328,49.961248,Rain (2001)
2,1023,49.702010,Winnie the Pooh and the Blustery Day (1968)
3,3658,49.699293,Quatermass and the Pit (1967)
4,6853,49.470448,Campus Man (1987)
5,899,11.350601,Singin' in the Rain (1952)
6,7223,10.063462,D.O.A. (1950)
7,5485,10.048250,Tadpole (2002)
8,4043,9.673469,At Close Range (1986)
9,913,8.573215,"Maltese Falcon, The (1941)"


The ```topN``` dataframe represents the top N recommendation list a user. These items are sorted in decreasing order of similarities with $I_u$.

<b>Observation</b> : The recommended items are the most similar to the set $I_u$ of items already purchased by the user.

## Top N recommendation with predictions

Before recommending the previous list to the user, we can go further and predict the ratings the user would have given to each of these items, sort them in descending order of prediction and return the reordered list as the new top N recommendation list.

### Rating prediction

As stated earlier, the predicted rating $\hat{r}_{u,i}$ for a given user $u$ on an item $i$ is obtained by aggregating ratings given by $u$ on items similar to $i$ as follows:

<center>
$
\large
 \hat{r}_{u,i}=\frac{\sum_{j\in S^{(i)}}r_{u,j}\cdot w_{i,j}}{\sum_{j\in S^{(i)}}|w_{i,j}|}
$
</center>

In [362]:
def predict(userid, itemid):
    """
    Make rating prediction for user userid on item itemid
    
    :param
        - userid : id of the active user
        - itemid : id of the item for which we are making prediction
        
    :return
        - r_hat : predicted rating
    """
    
    # index of itemid in the nearest neighbors space
    idx = itemids_to_idx[itemid]
    
    # Get items similar to item itemid with their corresponding similarities
    i_neighbors = neighbors[idx][1:]
    i_similarities = similarities[idx][1:]
    
    # initialize denominator
    W = 0
    
    # initialize numerator
    weighted_sum = 0
    
    for iid in i_neighbors:
        # get rating of userid on iid if exists
        if ((ratings.userid==userid) & (ratings.itemid==iid)).any():
            r = ratings[(ratings.userid==userid) & (ratings.itemid==iid)].rating.values[0]
            
            # get similarity between iid and itemid
            w = i_similarities[list(i_neighbors).index(iid)]
            
            # update denominator
            W = W + abs(w)
            
            # update numerator
            weighted_sum = weighted_sum + r * w
            
    if weighted_sum == 0:
        r_hat = mean[mean.userid==userid].rating.values[0]
    else:    
        # predicted rating
        r_hat = weighted_sum / W
    
    return r_hat

Let's check if this function predicts reasonable scores ...

In [350]:
ratings.head()

,userid,itemid,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [351]:
predict(1,1)

4.66814165339893

In [367]:
predict(1,3)

4.366379310344827

In [368]:
predict(1,47)

4.366379310344827

Now let's use our ```predict()``` function to predict what ratings the user would have given to the previous top-$N$ list and return the reorganised list (in decreasing order of predictions) as the new top-$N$ list

In [328]:
def topN_with_prediction(userid):
    """
    :param
        - userid : id of the active user
    
    :return
        - topN_list : initial topN recommendations returned by the function item2item_topN
        - topN_predict : topN recommendations reordered according to rating predictions
    """
    # make top N recommendation for the active user
    topN_list = item2item_topN(userid)
    
    # get list of items of the top N list
    itemids = topN_list.itemid.to_list()
    
    predictions = []
    
    # make prediction for each item in the top N list
    for itemid in itemids:
        r = predict(userid, itemid)
        
        predictions.append((itemid,r))
    
    predictions = pd.DataFrame(predictions, columns=['itemid','prediction'])
    
    # merge the predictions to topN_list and rearrange the list according to predictions
    topN_predict = pd.merge(topN_list, predictions, on='itemid', how='inner')
    topN_predict = topN_predict.sort_values(by=['prediction'], ascending=False)
    
    return topN_list, topN_predict

Now, let's make recommendation for user 514 and compare the two list

In [372]:
topN_list, topN_predict = topN_with_prediction(userid=514)

In [373]:
topN_list

,itemid,similarity_with_Iu,title
0,2470,50.154702,Crocodile Dundee (1986)
1,5328,49.961248,Rain (2001)
2,1023,49.702010,Winnie the Pooh and the Blustery Day (1968)
3,3658,49.699293,Quatermass and the Pit (1967)
4,6853,49.470448,Campus Man (1987)
5,899,10.350601,Singin' in the Rain (1952)
6,7223,9.063462,D.O.A. (1950)
7,5485,9.048250,Tadpole (2002)
8,4043,8.673469,At Close Range (1986)
9,913,7.573215,"Maltese Falcon, The (1941)"


In [374]:
topN_predict

,itemid,similarity_with_Iu,title,prediction
16,3514,6.706833,Joe Gould's Secret (2000),5.000000
11,848,7.295336,"Spitfire Grill, The (1996)",5.000000
18,1816,6.507557,Two Girls and a Guy (1997),4.742020
0,2470,50.154702,Crocodile Dundee (1986),4.441869
19,3445,6.379549,Eyes of Laura Mars (1978),4.211818
7,5485,9.048250,Tadpole (2002),4.169280
12,3497,7.295336,Max Dugan Returns (1983),4.004657
25,257,5.599141,Just Cause (1995),4.000000
23,3992,6.060868,MalÃ¨na (2000),4.000000
5,899,10.350601,Singin' in the Rain (1952),4.000000


As you will have noticed, the two lists are sorted in different ways. The second list is organized according to the predictions made for the user.

<b>Note</b>: When making predictions for user $u$ on item $i$, user $u$ may not have rated any of the $k$ most similar items to i. In this case, we consider the mean rating of $u$ as the predicted value.

## References

1. George Karypis (2001)<a href="https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.554.1671&rep=rep1&type=pdf">Evaluation of Item-Based Top-N Recommendation Algorithms</a>
2. Sarwar et al. (2001) <a href="https://dl.acm.org/doi/10.1145/371920.372071"> Item-based collaborative filtering recommendation algorithms</a> 
3. Michael D. Ekstrand, et al. (2011). <a href="https://dl.acm.org/doi/10.1561/1100000009"> Collaborative Filtering Recommender Systems</a>
4. J. Bobadilla et al. (2013)<a href="https://romisatriawahono.net/lecture/rm/survey/information%20retrieval/Bobadilla%20-%20Recommender%20Systems%20-%202013.pdf"> Recommender systems survey</a>
5. Greg Linden, Brent Smith, and Jeremy York (2003) <a href="https://www.cs.umd.edu/~samir/498/Amazon-Recommendations.pdf">Amazon.com Recommendations : Item-to-Item Collaborative Filtering</a>

## Author

<a href="https://www.linkedin.com/in/carmel-wenga-871876178/">Carmel WENGA</a>, Applied Machine Learning Research Engineer | <a href="https://shoppinglist.cm/fr/">ShoppingList</a>, Nzhinusoft